# HyDE

쿼리에 대한 hallucinated answer 의도적으로 생성해서 , 그 답안과 비슷한 문서 찾기.

In [2]:
!pip install llama-index datasets openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.

In [3]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:··········


In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
import pprint
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [5]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1284879 [00:00<?, ? examples/s]

In [6]:
data = ds.to_pandas()

In [7]:
data

,text,source,token_count,__index_level_0__
0,사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 ...,oscar2201,3348,0
1,“아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노...,oscar2201,1427,1
2,일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다...,oscar2201,2458,2
3,"재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 ...",oscar2201,2838,3
4,"상급자의 범죄와 비리, 부패를 하급자에게 돌리는 것으로 따지면 타의추종을 불허하는 ...",oscar2201,1628,4
5,최근 언론보도에 의하면 이재현 CJ그룹 회장이 지난해 말 두 자녀에게 증여하였던 주...,oscar2201,1366,5
6,"나는 노무현의 시대를 살지 않았다. 그러니까, 나는 이 땅의 생명체로 살아있긴 했지...",oscar2201,2017,6
7,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,oscar2201,1421,7
8,"안녕하세요. 한화솔루션입니다. 지난주, 슬기로운 솔루션 직장생활 2탄에 이어 이번엔...",oscar2201,2143,8
9,캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀...,oscar2201,1104,9


In [8]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [9]:
index = VectorStoreIndex.from_documents(docs)

In [10]:
pprint.pprint(data.iloc[11,0])

('얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재미는 물론 코딩을 통해 직접 움직이고 다양한 게임도 '
 '즐기고 즐거움을 주는 코딩 브릭인데요. 아이들에게 쉽고 재밌게 코딩 교육을 해 줄 수 있기 때문에 많은 사람들이 선물로 혹은 코딩 교육의 '
 '목적으로 선택하고 있습니다. 지난번에는 움직이는 로봇 버니를 통해 간단한 레고 부스트 소개와 코딩의 기초를 보여 드렸는데요. 오늘은 '
 '귀염둥이 고양이 프랭키를 통해 교감하는 애완동물을 만들어 보도록 하겠습니다. 레고 부스트는 하나로 5가지를 만들 수 있는 5-in-1 '
 '제품이라 다양한 방식으로 체험할 수 있기 때문에 단순히 만들고 분해하기를 반복하는 것보다 더 흥미 유발이 되더라구요. 버니를 분해해야 '
 '한다고 하니 많이 아쉬워 하긴 하더라구요. 하나를 만들고 나면 다시 분해해서 다른 것을 만들어야 하기 때문에 부품의 분실이나 보관에 '
 '신경을 써야 하는데요. 이런저런 방법을 생각하다가 일단 락앤락 보관 통을 이용해 봤습니다. 하나만 있어도 꽉 차게 들어가긴 하는데, 너무 '
 '꽉 차면 만들 때 불편하고 완전히 분해하면 한 통으로 모자라기 때문에 2개 정도 준비해 주는 게 좋습니다. 저는 기본적으론 색깔별로 '
 '분리하고, 연결 부품 따로 넓은 것 혹은 긴것 이런 식으로 모아 놓으니까 찾기도 편하고 만들기가 쉽더라구요. 레고 부스트는 다른 레고와 '
 '다르게 스마트폰이나 태블릿을 이용해 설명서를 보면서 만들도록 되어 있습니다. 나중에 다 만들고 나면 코딩까지 연계하기 위한 방법인데, '
 '자유롭게 확대와 축소가 가능하고 훼손 가능성도 적으니 다른 레고도 이렇게 설명서를 제공해 주면 좋겠다는 생각이 들 정도로 상당히 '
 '편리합니다. 물론 스마트폰보다는 화면이 큰 태블릿이 보기 편한데, 안드로이드와 아이폰 모두 지원하고 버전별로 호환 가능 여부가 다르니 '
 '아래 링크에서 확인할 수 있습니다. 지난번 움직이는 로봇 버니를 만들어 봐서 그런지 이번에는 

In [11]:
# Example: 청크 컨텍스트를 기반으로 2차적인 Reasoning이 필요할 때
query_str = "프랭키가 누구한테 인기가 없을까?"

In [12]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"{response}"))

프랭키는 일본 축구에 대한 한국인의 비판적인 시각 속에서 인기가 없을 것으로 보인다. 한국에서 일본 축구에 대한 사실과 다른 인식이 존재하며, 이는 한국인들이 일본 축구에 대해 느끼는 열등감과 관련이 있을 수 있다.

In [13]:
# HyDE 적용
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"{response}"))

프랭키는 특히 남자아이들에게 인기가 없을 것 같습니다.

In [14]:
# HyDE로 생성된 가상의 문서 확인
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'프랭키는 학교에서 인기가 없는 학생 중 한 명이다. 그의 외모는 일반적이지 않고, 독특한 스타일을 가지고 있어 다른 학생들과 잘 어울리지 못한다. 또한, 프랭키는 내성적인 성격을 가지고 있어 친구를 사귀는 데 어려움을 겪는다. 그는 종종 혼자 있는 모습을 보이며, 다른 학생들이 그를 따돌리거나 무시하는 경우도 많다. \n\n프랭키는 취미로 만화를 그리거나 비디오 게임을 즐기지만, 이러한 관심사는 주류 문화와는 거리가 있어 친구들과의 대화에서 소외감을 느끼곤 한다. 또한, 그는 자신이 좋아하는 것에 대해 열정적으로 이야기하지만, 다른 학생들은 그에 대해 관심을 가지지 않는 경우가 많아 대화가 잘 이어지지 않는다. \n\n이러한 이유들로 인해 프랭키는 학교에서 인기가 없고, 종종 외로움을 느끼며 지낸다. 그럼에도 불구하고 그는 자신의 개성을 잃지 않으려 노력하며, 언젠가는 진정한 친구를 만날 수 있기를 바라고 있다.'

In [15]:

pprint.pprint(data.iloc[12,0])

('대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우를 축적해왔다. 신도시 개발사에 1세대(분당, 일산, '
 '평촌 등), 2세대(광교, 판교, 위례, 동탄 등) 등의 수식어가 붙을 정도다. 많은 포토폴리오를 쌓은 한국의 신도시 건설업체들은 수십년 '
 '전부터 개발도상국으로 눈을 돌렸다. 그런데 어디에도 제대로된 한국적 신도시 성공담이 들리지 않는다. 오히려 고전을 면치못하는 게 '
 '현실이다. 해외에서 한국 건설사가 가장 많이 진출한 베트남에 현재 3개의 신도시 (D사의 하노이 레이크뷰시티, P사의 하노이 스플랜도라, '
 'G사의 호치민시 나베신도시)는 사업시작한지 10년 – 20년이 경과되었음에도 불구하고 아직 첫삽을 뜨지도 못했거나 1단계에서 '
 '지지부진하다. 왜 그럴까? 이 해답에 자신있게 대답을 할 수 없다면 해외신도시 사업은 재고하기 바란다. 주저앉은 베트남 신도시 사업을 '
 '분석해보면 다음과 같은 문제점이 발견된다. 첫째, 로컬파트너와의 긴밀한 협력 부재로 인허가 및 보상에서 수년이 지체되는 과정을 겪었다. '
 '베트남의 인허가 과정은 한국과는 전혀 다르다. 오랜 시간 정부 관계자와 친분을 쌓아야 하고 보이지 않는 전 방위적인 로비 문화가 있어 그 '
 '구조를 파악하는 대도 시간이 꽤 걸린다. 이쪽 구조에 밝고 신뢰가 쌓인 로컬파트너 없이는 불가능하다고 보면 된다. 한국 건설사들이 신도시 '
 '사업 시 100% 단독 지분구조를 언론을 통해 자랑처럼 얘기하지만 결국 그것이 발목을 잡는다. 어설픈 대행사를 끼고 단독으로 진행하다 '
 '인허가 과정에서 스케줄이 지연되고 더 많은 인허가 자금이 들어가는 등 매우 비생산적이고 비효율적인 사업으로 흘러간다. 토지 보상도 '
 '마찬가지이다. 사회주의라 정부가 공권력 동원해서 대거 강제 철거해버릴 것 같지만 민주주의보다 더 힘든것이 베트남에서의 보상 및 주민 이주 '
 '절차이다. 개발업자가 정부에 보상금을 지불하고 정부가 주민과 적정선에서 협상하면서 이주와

In [16]:
# Example: 청크 컨텍스트와 간접적으로 유사한 표현으로 물어봤을 때
query_str = "분당과 같은 도시가 외국에서도 통하려면 어떻게해야되?"

In [17]:
#Naive RAG
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"{response}"))

외국에서 분당과 같은 도시가 성공적으로 자리 잡기 위해서는 해당 지역의 사람, 문화, 시장, 관계, 제도 등을 깊이 이해하는 것이 중요합니다. 또한, 시행과 시공이 분리되어야 하며, 거시적 및 미시적 경제상황에 따라 유연하게 대응할 수 있는 주택상품을 기획해야 합니다. 로컬의 특성을 잘 반영하고, 현지의 요구에 맞춘 개발 전략을 세우는 것이 성공의 열쇠가 될 것입니다.

In [18]:
#HyDE
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"{response}"))

분당과 같은 도시가 외국에서도 성공적으로 자리잡기 위해서는 현지 문화를 깊이 이해하고, 로컬 파트너와의 긴밀한 협력이 필수적입니다. 또한, 각국의 인허가 절차와 시장 구조를 충분히 파악해야 하며, 이를 통해 신뢰를 구축하는 것이 중요합니다. 단독 지분 구조에 대한 고집을 버리고, 현지의 특성을 반영한 협력 모델을 채택해야 합니다. 이러한 접근 방식이 성공적인 도시 개발의 기초가 될 것입니다.

In [19]:
# 가상의 문서 확인
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'분당과 같은 도시가 외국에서도 통하려면 여러 가지 요소를 고려해야 합니다. 첫째, 도시의 인프라와 교통 시스템을 국제 기준에 맞춰 발전시켜야 합니다. 예를 들어, 대중교통의 편리함과 접근성을 높이고, 공항과의 연결성을 강화하여 외국인들이 쉽게 방문할 수 있도록 해야 합니다.\n\n둘째, 문화와 관광 자원을 다양화하여 외국인들에게 매력적인 요소를 제공해야 합니다. 지역의 역사, 전통, 음식 등을 홍보하고, 국제적인 행사나 축제를 개최하여 외국인 관광객을 유치할 수 있습니다. 또한, 다국적 기업과의 협력을 통해 외국인 투자자들에게 매력적인 비즈니스 환경을 조성하는 것도 중요합니다.\n\n셋째, 외국어 교육과 다문화 이해를 증진시켜야 합니다. 외국인 주민이나 관광객들이 편안하게 소통할 수 있도록 영어 및 다른 주요 언어로 서비스와 정보를 제공해야 하며, 지역 주민들도 다양한 문화에 대한 이해를 높이는 교육 프로그램에 참여할 수 있어야 합니다.\n\n마지막으로, 도시의 브랜드 이미지를 강화하고, 글로벌 네트워크를 구축하여 외국과의 교류를 활성화해야 합니다. 이를 통해 분당이 국제적으로 인지도를 높이고, 외국인들에게 매력적인 도시로 자리 잡을 수 있을 것입니다. 이러한 노력들이 결합되어 분당과 같은 도시가 외국에서도 통할 수 있는 기반을 마련할 수 있습니다.'

In [20]:

pprint.pprint(data.iloc[15,0])

('법무법인 엘케이비앤파트너스(엘케이비)는 코로나19 시대 의뢰인과 함께하는 전문 법률서비스를 제공하고 있다. 서울 서초동에선 엘케이비를 '
 '두고 “판검사나 변호사 등 프로들이 자기 사건을 맡기는 로펌”이라는 평이 많다. 2012년 설립 이래 탄탄한 송무 역량으로 법률시장을 '
 '주도하고 있다. 최근 엘케이비는 기업법무, 공정거래, 조세, 행정, 금융, 재건축·재개발, 노무, 지식재산권, 회생·파산, '
 'M&A(인수합병) 등 분야에서 종합적인 법률 솔루션을 제시하고 있다. 엘케이비의 가장 큰 강점은 베테랑 변호사들의 꼼꼼한 법률서비스. '
 '김강대 대표변호사는 “우리 로펌의 대표변호사들은 수임 사건에 대해 서면부터 재판 출석, 변론까지 모든 업무를 직접 챙긴다. 그 덕에 높은 '
 '승소율을 보이고 있다”며 “엘케이비는 최상의 법률서비스로 의뢰인과 동고동락하는 한편, 사회적공헌 활동(상자기사 참조)을 통해 국내 대표 '
 '로펌으로서 책임도 다할 것”이라고 말했다. 공정거래, 기업회생·M&A, 자본시장법, 은행·금융 분야 사건에서 최고 전문성을 지닌 엘케이비 '
 '대표변호사들을 만나 최근 법적 분쟁 양상과 승소 노하우를 들어봤다. 북한산 석탄을 러시아산으로 위장해 판매하던 A업체. 국내 B은행은 '
 '이런 사실을 모르고 해당 업체에 신용장(발행은행이 수입업자를 대신해 수출업자 앞으로 발행하는 문서)을 개설해줬다. 또 다른 C은행이 '
 '수익자(수출업자)로부터 신용장을 매입했다. 뒤늦게 해당 업체가 수입한 석탄이 미국의 대북 세컨더리 보이콧(secondary '
 'boycott) 대상인 북한산(産)으로 밝혀졌다. “특약에 따라 신용장 대금을 지급할 수 없다”는 B은행과 “국제규범인 '
 '‘신용장통일규칙’의 지급 의무를 지켜라”는 C은행 간 법적 공방이 벌어졌다. B은행 측 변호인으로 사건을 맡았던 이화용 '
 '대표변호사(사법연수원 26기)는 비슷한 사례를 찾았으나 여의치 않았다. 정치적으로 민감한 국제사회의 대북제재와 은행 간 이해관계가 얽

In [21]:
# 필요 없는 경우: 질문을 잘 했을 때
query_str = "법인회생을 신청한 기업이 몇고이고, 2019년 대비 몇% 줄었어?"

In [22]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"{response}"))

법인회생을 신청한 기업은 892곳이며, 이는 2019년 대비 11% 줄어든 수치입니다.

In [23]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"{response}"))

법인회생을 신청한 기업은 892곳이며, 2019년 대비 11% 줄었습니다.

In [24]:
# HyDE로 생성된 가상의 문서 확인
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'법인회생을 신청한 기업의 수는 2023년 기준으로 약 1,200개로 집계되었습니다. 이는 2019년에 비해 약 30% 감소한 수치입니다. 2019년에는 약 1,700개의 기업이 법인회생을 신청했으나, 이후 경제 회복과 정부의 지원 정책 등이 효과를 발휘하면서 신청 기업 수가 줄어든 것으로 분석됩니다. 특히, 코로나19 팬데믹 이후 많은 기업들이 어려움을 겪었지만, 2021년부터 시작된 경기 회복세와 함께 기업들의 재정 상태가 개선되면서 법인회생 신청이 감소한 것으로 보입니다. 이러한 변화는 기업들이 자생력을 키우고, 경영 효율성을 높이기 위한 노력의 일환으로 해석될 수 있습니다.'